In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
import numpy as np

In [7]:
print(tf.__version__)

1.9.0


In [8]:
imdb = keras.datasets.imdb

In [9]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 2s 0us/step


In [19]:
word_index = imdb.get_word_index()

1646592/1641221 [==============================] - 2s 1us/step


In [59]:
word_index = {k:(v+3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index  = dict([(val,key) for key, val in word_index.items()])

In [105]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i,'?') for i in text])

In [108]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                       value=word_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=256)

In [109]:
len(train_data[0]), len(test_data[0])

(256, 256)

In [116]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [117]:
model.compile(optimizer=tf.train.AdamOptimizer(),
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [118]:
x_val = train_data[:10000]
x_train = train_data[10000:]

y_val = train_labels[:10000]
y_train = train_labels[10000:]

In [119]:
history = model.fit(x_train,y_train,epochs=40,batch_size=512,
                   validation_data=(x_val,y_val),
                   verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 2s 116us/step - loss: 0.6921 - acc: 0.5143 - val_loss: 0.6918 - val_acc: 0.5250
Epoch 2/40
15000/15000 [==============================] - 1s 45us/step - loss: 0.6906 - acc: 0.5307 - val_loss: 0.6901 - val_acc: 0.5344
Epoch 3/40
15000/15000 [==============================] - 1s 45us/step - loss: 0.6889 - acc: 0.5384 - val_loss: 0.6885 - val_acc: 0.5503
Epoch 4/40
15000/15000 [==============================] - 1s 44us/step - loss: 0.6869 - acc: 0.5585 - val_loss: 0.6859 - val_acc: 0.5566
Epoch 5/40
15000/15000 [==============================] - 1s 44us/step - loss: 0.6830 - acc: 0.5966 - val_loss: 0.6820 - val_acc: 0.5995
Epoch 6/40
15000/15000 [==============================] - 1s 45us/step - loss: 0.6779 - acc: 0.6447 - val_loss: 0.6764 - val_acc: 0.6429
Epoch 7/40
15000/15000 [==============================] - 1s 45us/step - loss: 0.6707 - acc: 0.6955 - val_loss: 0.6687 - val_ac

In [120]:
results = model.evaluate(test_data,test_labels)
print(results)

25000/25000 [==============================] - 0s 19us/step
[0.306352166261673, 0.87376]


In [121]:
history_dict = history.history
history_dict.keys()

dict_keys(['acc', 'val_loss', 'loss', 'val_acc'])

In [125]:
import matplotlib.pyplot as plt
acc = history_dict['acc']
val_loss = history_dict['val_loss']
loss = history_dict['loss']
val_acc = history_dict['val_acc']

epochs = range(1, len(acc)+1)

In [126]:
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validataion Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [129]:
plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validataion Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()